In [1]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [3]:
import torch
print(torch.__version__)

1.4.0


In [4]:
# Construct a 5x3 matrix, uninitialized
x = torch.empty(5, 3)

# Construct a randomly initialized matrix   
x = torch.rand(5, 3)

# Construct a matrix filled zeros and of dtype long
x = torch.zeros(5, 3, dtype=torch.long)   

# Construct a tensor directly from data   
x = torch.tensor([5.5, 3])


In [5]:
x = torch.rand(5, 3)   
y = torch.rand(5, 3)   
# Syntax 1
z = x + y  
# Syntax 2
z = torch.empty(5, 3)
torch.add(x, y, out=z)

tensor([[0.9431, 0.4367, 1.0679],
        [0.2640, 1.0220, 1.1718],
        [0.8701, 1.3345, 1.2007],
        [0.9383, 0.7177, 0.5465],
        [0.6327, 0.9497, 1.2230]])

In [6]:
a = torch.ones(5) # Torch Tensor   
b = a.numpy() # NumPy Array


In [7]:
import numpy as np
a = np.ones(5) # NumPy Array
b = torch.from_numpy(a) # Torch Tensor


In [8]:
x = torch.ones(2, 2, requires_grad=True)   # or
x = torch.ones(2, 2)  
x.requires_grad_(True) # in-place


tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

In [9]:
y = x + 2
z = y * y * 3  
out = z.mean()

out.backward()

In [10]:
print(x.requires_grad)



True


In [12]:
with torch.no_grad():
   # Do operations on x
   x = x +1

In [13]:
import torch  
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(  [transforms.ToTensor(),
               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                 download=True, transform=transform)  
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                 download=True, transform=transform)  
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
 shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [14]:
testloader.dataset.classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [15]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [16]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [17]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [18]:
for epoch in range(2): # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):   
        # get the inputs
        inputs, labels = data
        # zero the parameter gradients  
        optimizer.zero_grad()
        # forward + backward + optimize  
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()  
        optimizer.step()
        # print statistics  
        running_loss += loss.item()
        if i % 2000 == 1999: # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f'%
                (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0


[1,  2000] loss: 2.158
[1,  4000] loss: 1.799
[1,  6000] loss: 1.662
[1,  8000] loss: 1.574
[1, 10000] loss: 1.487
[1, 12000] loss: 1.439
[2,  2000] loss: 1.388
[2,  4000] loss: 1.344
[2,  6000] loss: 1.341
[2,  8000] loss: 1.326
[2, 10000] loss: 1.282
[2, 12000] loss: 1.273


In [19]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:  
        images, labels = data  
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images:	%d'%
	(100 * correct / total))


Accuracy of the network on the 10000 test images:	56
